In [243]:
import os
import sys
import struct
import numpy
import cryptography.hazmat
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
#from cryptography.hazmat.primitives.ciphers import Cipher
#from cryptography.hazmat.primitives.ciphers.algorithms import AES
#from cryptography.hazmat.primitives.ciphers.modes import CTR # Might use, has parallel encryption
from cryptography.hazmat.backends import default_backend

In [244]:
def Create256BitKDF(salt):
    return PBKDF2HMAC(
        algorithm=hashes.SHA512_256(), # Faster than 256 on 64bit machines
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )

In [245]:
def DeriveKeyFromString(strIn):
    
    kdf = Create256BitKDF(os.urandom(32))
    
    # Cast strIn into something usable
    return kdf.derive(strIn.encode())

In [246]:
def DeriveKeyFromStringWithSalt(strIn, salt):
    
    kdf = Create256BitKDF(salt)
    
    # Cast strIn into something usable
    return kdf.derive(strIn.encode())

In [247]:
def DeriveKeyFromBytesWithSalt(data, salt):
    
    kdf = Create256BitKDF(salt)
    
    # Cast strIn into something usable
    return kdf.derive(data)

In [248]:
class FileMetadata:
    def __init__(self, salt=os.urandom(32), length = 0):
        self.check = "LING".encode()
        self.salt = salt
        self.fileLength = length
        self.sane = True
        self.nonce = os.urandom(16)
        
    def setSalt(self, salt):
        self.salt = salt
        
    def setFileLength(self, length):
        self.fileLength = length
        
    def getFileLength(self):
        return self.fileLength
    
    def getSalt(self):
        return self.salt
    
    def getAESKey(self, passphrase):
        return DeriveKeyFromBytesWithSalt(passphrase, self.salt)
    
    def getAESKeyFromString(self, passphrase):
        return DeriveKeyFromStringWithSalt(passphrase, self.salt)
    
    def getMetadataAsStruct(self):
        return struct.pack("4sl32s16s", self.check, self.fileLength, self.salt, self.nonce)
    
    def getNonce(self):
        temp = self.nonce
        self.nonce = os.urandom(16)
        return temp
    
    def setMetadataFromStruct(self, st):
        self.check, self.fileLength, self.salt, self.nonce = struct.unpack("4sl32s16s", st)
        
        if self.check != "LING":
            self.sane = False
            
    def isSane(self):
        return self.sane

In [276]:
class File:
    def __init__(self, filename, key, metadata=FileMetadata()):
        self.meta = metadata
        self.data = b""
        self.filename = filename
        self.writeIndex = 0
        self.fKey = key
        
        if not os.path.exists(filename):
            raise FileNotFoundError()
            
    def _LoadMeta(self, header):
        self.meta.setMetadataFromStruct(header)
        
    def _Encrypt(self, data):
        ctx = AESGCM(self.meta.getAESKeyFromString(self.fKey))
        return ctx.encrypt(self.meta.getNonce(), data, None)
    
    def _Decrypt(self, data):
        ctx = AESGCM(self.meta.getAESKeyFromString(self.fKey))
        return ctx.decrypt(self.meta.getNonce(), data, None)
        
    def Write(self, data):
        # Size
        nNotOverwritten = self.writeIndex
        nOverwritten = len(data)
        
        print(self.data[0:nNotOverwritten])
        
        totalSize = nNotOverwritten + nOverwritten
        
        writeData = numpy.zeros(totalSize, dtype=(numpy.void, 1))
        
        # Copy to writeData
        # Copy not overwritten
        if nNotOverwritten:
            print(writeData)
            print(self.data[0:nNotOverwritten])
            writeData[0:nNotOverwritten] = self.data[0:nNotOverwritten]
            
        print(writeData)
        writeData[nNotOverwritten: nNotOverwritten + nOverwritten] = data[0:nOverwritten]
        
        print(writeData)
        sys.exit(1)
            
        # Write to disk!
        with open(self.filename, "wb") as f:
            f.write(self.meta.getMetadataAsStruct())
            f.write(self._Encrypt(self.data))
        
    def Read(self):
        if self.data:
            return self.data
        else:
            # Load the file
            with open(self.filename, "rb") as f:
                self.data = f.read()
                
            try:
                self._LoadMeta(self.data[0:64])
                self.data = self._Decrypt(self.data[64:])
                self.writeIndex = len(self.data)
            except Exception as e:
                print(e)
                # No Encrypted Yet
                pass
                
            return self.data
        

In [277]:
md = File("out.md", "hi.php")
print(md.Read())
md.Write("")

b'WERTTTE\n'
b'WERTTTE\n'
[b'\x00' b'\x00' b'\x00' b'\x00' b'\x00' b'\x00' b'\x00' b'\x00']
b'WERTTTE\n'
[b'\x57' b'\x57' b'\x57' b'\x57' b'\x57' b'\x57' b'\x57' b'\x57']
[b'\x57' b'\x57' b'\x57' b'\x57' b'\x57' b'\x57' b'\x57' b'\x57']


SystemExit: 1

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
